In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from scipy.stats import norm
import numbers
from pandas.api.types import is_numeric_dtype, is_bool_dtype
from osm import get_pois
#from scrape_bienici import scrape_pages

In [2]:
df_raw = pd.read_csv("bienIci_2458.csv")

In [3]:
df_raw.head()

,transactionType,rentWithoutCharges,charges,energyPerformanceDiagnosticDate,title,heating,district.name,hasTerrace,greenhouseGazClassification,greenhouseGazValue,...,bedroomsQuantity,descriptionTextLength,postalCodeForSearchFilters,price,city,blurInfo.type,status.onTheMarket,district.libelle,blurInfo.centroid.lon,hasElevator
0,rent,1771.0,NaN,2023-09-19,Exclusivité- Appartement meublé- Paris XV,électricité individuel,Paris 15e Arrondissement - Cambronne - Garibaldi,NaN,B,6.0,...,1.0,667,75015,1771.0,Paris 15e,disk,True,Cambronne - Garibaldi,2.303488,NaN
1,rent,1500.0,450.0,2024-06-25,2 pièces avec terrasse - Paris 15,radiateur gaz collectif,Paris 15e Arrondissement - Alleray - Procession,True,C,29.0,...,1.0,474,75015,1950.0,Paris 15e,disk,True,Alleray - Procession,2.304538,True
2,rent,13000.0,NaN,2025-07-06,Appartement exceptionnel- Paris VIII,NaN,Paris 8e Arrondissement - Hoche Friedland,True,D,33.0,...,4.0,850,75008,13000.0,Paris 8e,disk,True,Hoche Friedland,2.299560,NaN
3,rent,3800.0,NaN,2025-12-02,Appartement meublé - Paris 14ème,NaN,Paris 14e Arrondissement - Montsouris - Dareau,NaN,D,34.0,...,2.0,701,75014,3800.0,Paris 14e,disk,True,Montsouris - Dareau,2.335510,NaN
4,rent,866.0,NaN,2023-02-20,STUDIO-ILES SAINT LOUIS,NaN,Paris 4e Arrondissement - Les Iles,NaN,B,11.0,...,0.0,311,75004,866.0,Paris 4e,disk,True,Les Iles,2.354162,NaN


---

## A) Target / price variables (what you predict + normalize)
**Keys**
- `price`
- `transactionType`
- `rentWithoutCharges`
- `charges`
- `chargesMethod`
- `surfaceArea`
- `pricePerSquareMeter` *(optional as target or derived feature)*

**What to analyze / derive**
- `total_rent` (all-in):  
  - if available: `rentWithoutCharges + charges`  
  - else fallback: `price`
- `rent_net` (hors charges): `rentWithoutCharges`
- `rent_per_m2` (€/m²): `total_rent / surfaceArea` *(or use `pricePerSquareMeter` if reliable)*
- Handle missingness: when `charges` is 0 vs missing, etc.

---

In [4]:
# From FEATURE_INDEX, grouped by the A), B), C)… sections
GROUP_KEYS = {
    # A) Target / price variables
    "A_target_price": [
        "transactionType",
        "price",
        "rentWithoutCharges",
        "charges",
        "chargesMethod",
        "surfaceArea",
        "pricePerSquareMeter",
        "priceHasDecreased",
    ],

    # B) Location (coarse -> neighborhood -> micro + uncertainty)
    "B_location": [
        "city",
        "postalCode",
        "postalCodeForSearchFilters",
        "departmentCode",
        "district.code_insee",
        "district.insee_code",
        "district.name",
        "district.libelle",
        "addressKnown",
        "blurInfo.centroid.lon",
        "blurInfo.centroid.lat",
        "blurInfo.radius",
        "blurInfo.bbox",
        "blurInfo.type",
    ],

    # C) Size & functional layout
    "C_size_layout": [
        "propertyType",
        "surfaceArea",
        "roomsQuantity",
        "bedroomsQuantity",
    ],

    # D) Bathrooms / wet rooms
    "D_wet_rooms": [
        "bathroomsQuantity",
        "showerRoomsQuantity",
        "toiletQuantity",
    ],

    # E) Building / access constraints
    "E_building_access": [
        "floor",
        "hasElevator",
        "newProperty",
    ],

    # F) Unit comfort / livability
    "F_unit_comfort": [
        "exposition",
        "hasTerrace",
        "nothingBehindForm",
        "opticalFiberStatus",
        "with3dModel",
    ],

    # G) Energy performance & eco-comfort (DPE/GES)
    "G_energy": [
        "energyClassification",
        "energyValue",
        "minEnergyConsumption",
        "maxEnergyConsumption",
        "useJuly2021EnergyPerformanceDiagnostic",
        "energyPerformanceDiagnosticDate",
        "heating",
        "greenhouseGazClassification",
        "greenhouseGazValue",
    ],

    # H) Text-derived signals (for deducing missing amenities/condition)
    "H_text": [
        "title",
        "description",
        "descriptionTextLength",
    ],

    # I) Listing-quality proxies (optional)
    "I_listing_quality": [
        "photos",
        "isBienIciExclusive",
        "status.highlighted",
        "status.isLeading",
        "status.onTheMarket",
    ],

    # J) IDs & dedup (optional)
    "J_ids": [
        "id",
        "reference",
    ],
}


## Data cleaning rules (Price / Rent normalization)

### `transactionType`
- **Keep** only rows where `transactionType == "rent"`.
- **Drop** rows where `transactionType` is **missing** or **not rent**.

---

### `price` (displayed monthly price)
- Currently: **no N/A**.
- If `price` becomes missing in the future:
  - We must be able to reconstruct it from:
    - `rentWithoutCharges` and `charges`
  - Therefore, when `price` is missing, **do not allow** both `rentWithoutCharges` and `charges` to be missing.

---

### `chargesMethod`
- Fill missing values with: **`"UNKNOWN"`**.

---

### `rentWithoutCharges`
- If `rentWithoutCharges` is **missing** and `price` is **not missing**:
  - Fill `rentWithoutCharges = 0`.

---

### `charges`
- If `charges` is **missing** and `price` is **not missing**:
  - Fill `charges = 0`.

---

### `surfaceArea` and `pricePerSquareMeter`
- If `surfaceArea` is missing, try to deduce:
  - `surfaceArea = price / pricePerSquareMeter`
- **Drop** rows where **both** `surfaceArea` and `pricePerSquareMeter` are missing.

---

### G) `priceHasDecreased`
- Currently: **no N/A**.
- If missing values appear in the future:
  - Fill missing values with **`False`**.

In [7]:
def get_estimatedPrice(df : pd.DataFrame):
    """
    Rules :
    
    let:
    p = price
    r = rentWithoutCharges
    c = charges
    s = r + c
    es = estimatedPrice
    
    logics :
    if p doesn't exist, c/r doesn't exist :
    -> drop
    if p doesn't exist, c and r exist :
    -> es = s
    if p and s exist :
        p > s : es = p
        p < s : es = s
    """
    out_df = df.copy()
    p = out_df["price"]
    c = out_df["charges"]
    r = out_df["rentWithoutCharges"]
    #drop by rules
    m_dropna = (p.isna() & (c.isna() | r.isna() ))
    out_df = out_df[~m_dropna].copy()
    p = out_df["price"]
    c = out_df["charges"]
    r = out_df["rentWithoutCharges"]
    #compute estimated price
    s = c + r
    es = pd.concat([p,c,r], axis = 1).max(axis=1)
    m_rc_exist = r.notna() & c.notna()
    es.loc[m_rc_exist] = np.maximum(s.loc[m_rc_exist],es.loc[m_rc_exist])
    es = es.rename("estimatedPrice")
    out_df = out_df.join(es)
    return out_df.rename(columns = {"price" : "advertisedPrice"})

In [8]:
#get big delta between advertised and estimated, threshold = 10
df_v1 = get_estimatedPrice(df = df_raw)
df_v1.loc[
    abs(df_v1["advertisedPrice"]-df_v1["estimatedPrice"]) > 10
    ,["advertisedPrice","estimatedPrice"]]

,advertisedPrice,estimatedPrice


In [9]:
def sanitize_surface(df : pd.DataFrame):
    """
    let :
    s : surfaceArea
    pm2 : pricePerSquareMeter
    es : estimatedPrice
    rules :
    drop if both s & (pm2 or es area N/A)
    """
    out_df = df.copy()
    s = out_df["surfaceArea"]
    pm2 = out_df["pricePerSquareMeter"]
    es = out_df["estimatedPrice"]
    #drop rows by rules
    m_dropna = s.isna() & (pm2.isna() | es.isna())
    out_df = out_df[~m_dropna].copy()
    s = out_df["surfaceArea"]
    pm2 = out_df["pricePerSquareMeter"]
    es = out_df["estimatedPrice"]
    #deduce surfaceArea by pm2 and es
    try:
        computed_s = (es / pm2).where(s.isna())
        computed_s = computed_s.round().astype("Int64").rename("estimatedSurfaceArea")
    except Exception as e:
        raise RuntimeError("error at imputeSurface(df)") from e
    return out_df.join(computed_s)

In [10]:
df_v2 = sanitize_surface(df=df_v1)
df_v2.loc[df_v2["surfaceArea"].isna(),["surfaceArea","estimatedSurfaceArea"]]

,surfaceArea,estimatedSurfaceArea


In [11]:
def sanitize_pm2(df: pd.DataFrame):
    """
    let:
      s   : surfaceArea
      pm2 : pricePerSquareMeter
      es  : estimatedPrice

    rule:
      drop if pm2 is NA AND (s is NA OR es is NA)
      impute estimatedPricePerSquareMeter = es / s for rows where pm2 is NA
    """
    out_df = df.copy()

    s = out_df["surfaceArea"]
    pm2 = out_df["pricePerSquareMeter"]
    es = out_df["estimatedPrice"]

    # drop rows where pm2 missing and can't deduce it
    m_dropna = pm2.isna() & (s.isna() | es.isna())
    out_df = out_df.loc[~m_dropna].copy()

    s = out_df["surfaceArea"]
    pm2 = out_df["pricePerSquareMeter"]
    es = out_df["estimatedPrice"]

    try:
        computed_pm2 = (es / s).where(pm2.isna())
        computed_pm2 = computed_pm2.round(2).rename("estimatedPricePerSquareMeter")
    except Exception as e:
        raise RuntimeError("error at imputePm2(df)") from e

    return out_df.join(computed_pm2)

In [12]:
df_v3 = sanitize_pm2(df = df_v2)

In [13]:
len(df_v3)

2415

In [14]:
def impute_PriceHasDecreased(df : pd.DataFrame):
    out_df = df.copy()
    out_df["priceHasDecreased"] = out_df["priceHasDecreased"].fillna(False)
    return out_df

In [15]:
df_v4 = impute_PriceHasDecreased(df = df_v3)

In [16]:
def filter_transactionType(df : pd.DataFrame , ttype : str = "rent"):
    return df.loc[df["transactionType"] == ttype].copy()

In [17]:
def impute_chargesMethod(df: pd.DataFrame) -> pd.DataFrame:
    out_df = df.copy()
    out_df["chargesMethod"] = out_df["chargesMethod"].fillna("UNKNOWN")
    return out_df

In [18]:
df_v5 = impute_chargesMethod(df = df_v4)

## B) Location features (coarse → micro) + N/A handling rules

### B0) Goal
We want the model to always have **at least one usable location signal**.
If fine-grained geo is missing, we fall back to coarse location (postal code / city).

---

### B1) Columns in this group

**Coarse location (best for model + UI input)**
- `postalCodeForSearchFilters`
- `postalCode`
- `city`
- `departmentCode`

**Neighborhood identifiers (optional)**
- `district.code_insee`
- `district.insee_code`
- `district.name`
- `district.libelle`

**Micro-geo (optional, mainly for enrichment like POIs/transport)**
-> Done with osm.py get_pois(lan,lon,radius_m)
+ radius_m is defaulted as 500 due to latency issue. approximately 12sec per call.
- `blurInfo.centroid.lat`
- `blurInfo.centroid.lon`
- `blurInfo.bbox`
- `blurInfo.radius`
- `blurInfo.type`

**Precision proxy**
- `addressKnown`

---

### B2) Absolute requirement (drop rule)
Drop a row **only if all coarse location fields are missing**:

- `postalCodeForSearchFilters` is NA
- `postalCode` is NA
- `city` is NA

> If at least one of these exists, keep the row.

---

### B3) Safe imputations (no external lookup)
**1) Fill missing postalCodeForSearchFilters from postalCode**
```python
df["postalCodeForSearchFilters"] = df["postalCodeForSearchFilters"].fillna(df["postalCode"])


In [19]:
def sanitize_codePostal(df : pd.DataFrame):
    """
    Tier 1 (best): postalCodeForSearchFilters or postalCode present
            
    Tier 0: nothing present → drop
    """
    out_df = df.copy()
    out_df["postalCode"] = out_df["postalCode"].fillna(out_df["postalCodeForSearchFilters"])

    m_allNa = out_df[["postalCode","city"]].isna().all(axis=1)
    return out_df[~m_allNa].copy()

In [20]:
df_v6 = sanitize_codePostal(df=df_v5)
df_v6[GROUP_KEYS["B_location"]].isna().sum()

city                            0
postalCode                      0
postalCodeForSearchFilters      0
departmentCode                  0
district.code_insee             0
district.insee_code             0
district.name                   0
district.libelle                0
addressKnown                    0
blurInfo.centroid.lon           0
blurInfo.centroid.lat           0
blurInfo.radius               826
blurInfo.bbox                   0
blurInfo.type                   0
dtype: int64

In [21]:
def sanitize_layout(df : pd.DataFrame):
    """
    Drop propertyType if not "flat"
    Drop rows where roomsQuantity is NA (2 rows)
    
    Good rule baseline (France “T” logic):
    If roomsQuantity == 1 ⇒ bedrooms = 0 (studio)
    Else constrain: 0 ≤ bedrooms ≤ roomsQuantity - 1
    Practical default: bedrooms = roomsQuantity - 1 (works surprisingly well as a baseline)
    """
    out_df = df.copy()
    #dropping
    m_propertyTypeFlat = df["propertyType"] == "flat"
    m_roomsQuantityNa = df["roomsQuantity"].isna()
    m_validRoomsQuantity = df["roomsQuantity"] > 0
    out_df = out_df[m_propertyTypeFlat & ~m_roomsQuantityNa & m_validRoomsQuantity]
    
    #impute
    m_roomsQuantity1 = out_df["roomsQuantity"] == 1
    m_bedroomsQuantityNa = out_df["bedroomsQuantity"].isna()
    out_df.loc[m_roomsQuantity1 & m_bedroomsQuantityNa,"bedroomsQuantity"] = 0
    out_df.loc[~m_roomsQuantity1 & m_bedroomsQuantityNa,"bedroomsQuantity"] = out_df["roomsQuantity"] - 1
    
    return out_df

In [25]:
GROUP_KEYS.keys()

dict_keys(['A_target_price', 'B_location', 'C_size_layout', 'D_wet_rooms', 'E_building_access', 'F_unit_comfort', 'G_energy', 'H_text', 'I_listing_quality', 'J_ids'])

In [30]:
print("BEFORE :\n",df_v6[GROUP_KEYS["C_size_layout"]].isna().sum(),"\n")
df_v7 = sanitize_layout(df = df_v6)
print("AFTER :\n",df_v7[GROUP_KEYS["C_size_layout"]].isna().sum(),"\n")

BEFORE :
 propertyType          0
surfaceArea           0
roomsQuantity         2
bedroomsQuantity    191
dtype: int64 

AFTER :
 propertyType        0
surfaceArea         0
roomsQuantity       0
bedroomsQuantity    0
dtype: int64 



In [45]:
"""
Let's see the percentage of having number of showerRooms == bathroomsQuantity,
if the percentage is high compare to all known shower quant and bath quant,
we will use rule : if shower quant missing, set = bath quant, and vice-versa.
"""
m_showerRoomsQuantityNa = df_v7["showerRoomsQuantity"].isna()
m_bathroomsQuantityNa = df_v7["bathroomsQuantity"].isna()
m_equalBathShower = df_v7["bathroomsQuantity"] == df_v7["showerRoomsQuantity"]
m_bathGreaterShower = df_v7["bathroomsQuantity"] > df_v7["showerRoomsQuantity"]
m_bathLesserShower = df_v7["bathroomsQuantity"] < df_v7["showerRoomsQuantity"]

len_bathLesserShower = len(df_v7[m_bathLesserShower])
len_bathGreaterShower = len(df_v7[m_bathGreaterShower])
len_equals = len(df_v7[m_equalBathShower])
len_knowns = len(df_v7[~m_showerRoomsQuantityNa & ~m_bathroomsQuantityNa])
print("total knowns : ", len_knowns)
print("total equals : ", len_equals)
print(f"percentage : {((len_equals/len_knowns)*100):.2f}%")
print("total bath greater than shower :", len_bathGreaterShower, f"\npercentage : {(len_bathGreaterShower/len_knowns)*100:.2f}%")
print("total bath lesser than shower :", len_bathLesserShower, f"\npercentage : {(len_bathLesserShower/len_knowns)*100:.2f}%")


print("*"*10)
m_knowns = df_v7["bathroomsQuantity"].notna() & df_v7["showerRoomsQuantity"].notna()

diff = df_v7.loc[m_knowns, "bathroomsQuantity"] - df_v7.loc[m_knowns, "showerRoomsQuantity"]

# Case 1: bath > shower  -> diff > 0
median_diff_bath_greater = diff[diff > 0].median()

# Case 2: bath < shower  -> diff < 0
median_diff_bath_lesser = diff[diff < 0].median()

print("median(bath - shower) when bath > shower:", median_diff_bath_greater)
print("median(bath - shower) when bath < shower:", median_diff_bath_lesser)
print("median(|bath - shower|) when bath > shower:", diff[diff > 0].abs().median())
print("median(|bath - shower|) when bath < shower:", diff[diff < 0].abs().median())

total knowns :  506
total equals :  366
percentage : 72.33%
total bath greater than shower : 61 
percentage : 12.06%
total bath lesser than shower : 79 
percentage : 15.61%
**********
median(bath - shower) when bath > shower: 1.0
median(bath - shower) when bath < shower: -1.0
median(|bath - shower|) when bath > shower: 1.0
median(|bath - shower|) when bath < shower: 1.0


In [47]:
def sanitize_sanitary(df: pd.DataFrame):
    out_df = df.copy()

    b = out_df["bathroomsQuantity"]
    s = out_df["showerRoomsQuantity"]

    m_b_na = b.isna()
    m_s_na = s.isna()

    # --- stats-driven imputation between bath & shower ---
    # since ~72% equal and median gap is 1, the safest default is "copy the known value"
    
    # if shower missing but bath known -> shower = bath
    m = m_s_na & ~m_b_na
    out_df.loc[m, "showerRoomsQuantity"] = out_df.loc[m, "bathroomsQuantity"]

    # if bath missing but shower known -> bath = shower
    m = m_b_na & ~m_s_na
    out_df.loc[m, "bathroomsQuantity"] = out_df.loc[m, "showerRoomsQuantity"]

    # default toilet has at least 1
    out_df["toiletQuantity"] = out_df["toiletQuantity"].fillna(1)
    
    return out_df


In [48]:
GROUP_KEYS.keys()

dict_keys(['A_target_price', 'B_location', 'C_size_layout', 'D_wet_rooms', 'E_building_access', 'F_unit_comfort', 'G_energy', 'H_text', 'I_listing_quality', 'J_ids'])

In [51]:
print("BEFORE :\n",df_v7[GROUP_KEYS["D_wet_rooms"]].isna().sum(),"\n")
df_v8 = sanitize_sanitary(df = df_v7)
print("AFTER :\n",df_v8[GROUP_KEYS["D_wet_rooms"]].isna().sum(),"\n")

BEFORE :
 bathroomsQuantity      1288
showerRoomsQuantity    1463
toiletQuantity         1400
dtype: int64 

AFTER :
 bathroomsQuantity      844
showerRoomsQuantity    844
toiletQuantity           0
dtype: int64 



In [52]:
#E_building_access
#for elevator missing, due to advertiser bias, they would often hide what's not practical.
def impute_hasElevator(df: pd.DataFrame):
    out_df = df.copy()
    out_df["hasElevator"] = out_df["hasElevator"].fillna(False)
    return out_df

In [53]:
#for floors, we are leaving them as is, since there are not that much NA
print("BEFORE :\n",df_raw[GROUP_KEYS["E_building_access"]].isna().sum(),"\n")
df_v9 = impute_hasElevator(df = df_v8)
print("AFTER :\n",df_v9[GROUP_KEYS["E_building_access"]].isna().sum(),"\n")

BEFORE :
 floor          196
hasElevator    515
newProperty      0
dtype: int64 

AFTER :
 floor          194
hasElevator      0
newProperty      0
dtype: int64 



C:\Users\thaim\AppData\Local\Temp\ipykernel_62400\1132602765.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out_df["hasElevator"] = out_df["hasElevator"].fillna(False)


In [54]:
def estimate_extra_expense_energy(
    df: pd.DataFrame,
    surface_col: str = "surfaceArea",
    min_col: str = "minEnergyConsumption",
    max_col: str = "maxEnergyConsumption",
    energy_value_col: str = "energyValue",
    heating_col: str = "heating",
    elec_ep_to_final_coef: float = 2.3,   # set 1.9 if you want the 2026 rule
    price_per_kwh_elec: float | None = None,
    price_per_kwh_other: float | None = None,
    make_monthly: bool = True,
) -> pd.DataFrame:
    """
    Adds:
      - extra_energy_annual_min / avg / max
      - extra_energy_monthly_min / avg / max (optional)

    Priority:
      1) Use minEnergyConsumption/maxEnergyConsumption (assumed €/year)
      2) Fallback: if both missing AND price_per_kwh_* provided AND surface+energyValue exist,
         estimate €/year from kWhEP.
    """
    out = df.copy()

    # --- Tier 1: direct DPE annual € range (min/max) ---
    mn = pd.to_numeric(out.get(min_col), errors="coerce")
    mx = pd.to_numeric(out.get(max_col), errors="coerce")

    # unify: if only one bound exists, use it for all
    annual_min = np.where(mn.notna(), mn, mx)
    annual_max = np.where(mx.notna(), mx, mn)

    # avg if both exist else the known bound
    annual_avg = np.where(mn.notna() & mx.notna(), (mn + mx) / 2,
                  np.where(mn.notna(), mn,
                  np.where(mx.notna(), mx, np.nan)))

    # --- Tier 2: fallback from energyValue * surface * €/kWh ---
    need_fallback = pd.isna(annual_avg)

    if (price_per_kwh_elec is not None) and (price_per_kwh_other is not None):
        ev = pd.to_numeric(out.get(energy_value_col), errors="coerce")
        sa = pd.to_numeric(out.get(surface_col), errors="coerce")

        # annual primary energy kWhEP
        kwh_ep = ev * sa

        # crude electric detection from heating string
        heating = out.get(heating_col)
        heating_s = heating.fillna("").astype(str).str.lower() if heating is not None else pd.Series("", index=out.index)
        is_elec = heating_s.str.contains("elect|élect")

        # convert EP -> final for electricity; for others assume coef ~ 1
        kwh_final = np.where(is_elec, kwh_ep / elec_ep_to_final_coef, kwh_ep)
        price = np.where(is_elec, price_per_kwh_elec, price_per_kwh_other)

        est_annual = kwh_final * price

        # apply only where Tier 1 is missing
        annual_avg = np.where(need_fallback, est_annual, annual_avg)

        # if you only have a point estimate, set min=max=avg (you can widen later if you want)
        annual_min = np.where(need_fallback, est_annual, annual_min)
        annual_max = np.where(need_fallback, est_annual, annual_max)

    out["extra_energy_annual_min"] = annual_min
    out["extra_energy_annual_avg"] = annual_avg
    out["extra_energy_annual_max"] = annual_max
    out["extra_energy_is_missing"] = pd.isna(out["extra_energy_annual_avg"]).astype(int)

    if make_monthly:
        out["extra_energy_monthly_min"] = out["extra_energy_annual_min"] / 12
        out["extra_energy_monthly_avg"] = out["extra_energy_annual_avg"] / 12
        out["extra_energy_monthly_max"] = out["extra_energy_annual_max"] / 12

    return out


In [59]:
len(df_raw[df_raw["showerRoomsQuantity"].isna() & df_raw["bathroomsQuantity"].notna()])

622

In [88]:
len(cleaned_df)

2413

In [73]:
cleaned_df = pd.read_csv("test_cleanedDs.csv")

In [74]:
cleaned_df["computedToiletQuantity"].isna().sum()

np.int64(0)

In [80]:
cleaned_df.loc[cleaned_df["computedAvgEnergyConsumption"].isna(),"computedEnergyClassification"].value_counts()

computedEnergyClassification
NS    235
VI      2
Name: count, dtype: int64

In [95]:
df_raw.iloc[269]["postalCode"]

np.int64(92100)

In [ ]:
"""
df_raw.iloc[1659][["postalCode","title","description"]]
postalCode                                                 75014
title                                                         T4
"""